# LangChain Agents: Zero to Hero Guide

## Building Powerful Single-Agent Systems with Tools

**Objective:** This comprehensive notebook takes you from beginner to advanced LangChain Agent user. You'll learn how to create agents that reason, use tools, maintain memory, and handle complex tasks.

**Target Audience:** Software engineers from complete beginners to experts looking to master LangChain Agents.

---

## Table of Contents
1. [Introduction & Core Philosophy](#1-introduction--core-philosophy)
2. [Prerequisites & Setup](#2-prerequisites--setup)
3. [Core Concepts: The ReAct Loop](#3-core-concepts-the-react-loop)
4. [Your First Agent](#4-your-first-agent)
5. [Custom Tools with @tool Decorator](#5-custom-tools-with-tool-decorator)
6. [Structured Tools with Pydantic](#6-structured-tools-with-pydantic)
7. [Agent Memory & Chat History](#7-agent-memory--chat-history)
8. [Different Agent Types](#8-different-agent-types)
9. [Streaming & Real-time Output](#9-streaming--real-time-output)
10. [Error Handling & Debugging](#10-error-handling--debugging)
11. [Real-World Patterns](#11-real-world-patterns)
12. [Best Practices & Common Pitfalls](#12-best-practices--common-pitfalls)
13. [Conclusion & Next Steps](#13-conclusion--next-steps)

---

## 1. Introduction & Core Philosophy

### What is a LangChain Agent?

A **LangChain Agent** is an LLM-powered system that can reason about tasks and use tools to accomplish goals. Unlike simple chains that follow a fixed sequence, agents dynamically decide which actions to take based on the situation.

### Core Philosophy

| Principle | Description |
|-----------|-------------|
| **Reasoning** | Agents think before acting (ReAct pattern) |
| **Tool Use** | Agents extend LLM capabilities with external tools |
| **Autonomy** | Agents decide which tools to use and when |
| **Iteration** | Agents can retry and refine based on results |

### When to Use LangChain Agents?

✅ **Good for:**
- Tasks requiring external data (search, APIs, databases)
- Dynamic decision-making workflows
- Single-agent tool use scenarios
- Building blocks for larger systems

❌ **Consider alternatives when:**
- You need multi-agent collaboration (use AutoGen/CrewAI)
- You need complex state machines (use LangGraph)
- You have a fixed, predictable workflow (use simple chains)

### LangChain Agents vs Other Frameworks

```
┌────────────────────────────────────────────────────────────┐
│              Agent Framework Comparison                    │
├────────────────┬───────────────────────────────────────────┤
│ LangChain      │ Single agent, flexible tools, ReAct loop  │
│ LangGraph      │ State machines, complex control flow      │
│ AutoGen        │ Multi-agent conversations                 │
│ CrewAI         │ Role-playing agent crews                  │
└────────────────┴───────────────────────────────────────────┘
```

---

## 2. Prerequisites & Setup

### Requirements

- **Python 3.9+**
- **OpenAI API Key**
- **Tavily API Key** (optional, for web search)

### Installation

```bash
pip install langchain langchain-openai langchain-community
pip install tavily-python  # For web search tool
```

In [ ]:
# Install dependencies (uncomment to run)
# !pip install langchain langchain-openai langchain-community python-dotenv tavily-python

In [ ]:
import os
import warnings
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Verify API keys
openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

print("🔑 API KEY STATUS")
print("-" * 40)
print(f"OpenAI API Key: {'✅ Found' if openai_key else '❌ Missing'}")
print(f"Tavily API Key: {'✅ Found' if tavily_key else '⚠️ Optional'}")

if not openai_key:
    print("\n❌ Please add OPENAI_API_KEY to your .env file")

---

## 3. Core Concepts: The ReAct Loop

LangChain Agents use the **ReAct** (Reason + Act) pattern:

```
┌─────────────────────────────────────────────────────────────────┐
│                    The ReAct Loop                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│    User Query                                                   │
│        │                                                        │
│        ▼                                                        │
│    ┌─────────┐                                                  │
│    │ THOUGHT │  "I need to search for information..."          │
│    └────┬────┘                                                  │
│         │                                                       │
│         ▼                                                       │
│    ┌─────────┐                                                  │
│    │ ACTION  │  Call: search_tool("query")                     │
│    └────┬────┘                                                  │
│         │                                                       │
│         ▼                                                       │
│    ┌─────────────┐                                              │
│    │ OBSERVATION │  Tool returns: "Result data..."             │
│    └──────┬──────┘                                              │
│           │                                                     │
│           ▼                                                     │
│    ┌─────────┐                                                  │
│    │ THOUGHT │  "Now I have the info, I can answer..."         │
│    └────┬────┘                                                  │
│         │                                                       │
│         ▼                                                       │
│    Final Answer                                                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key Components

1. **LLM**: The "brain" that reasons and decides
2. **Tools**: Functions the agent can call
3. **Prompt**: Instructions telling the agent how to behave
4. **Agent Executor**: Orchestrates the ReAct loop

---

## 4. Your First Agent

Let's build a simple agent with web search capability.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Step 1: Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,  # More deterministic responses
)

print("✅ LLM initialized!")
print(f"   Model: gpt-4o")

In [ ]:
# Step 2: Set up tools
# The Tavily search tool provides web search capabilities

search_tool = TavilySearchResults(max_results=3)

tools = [search_tool]

print("✅ Tools configured!")
print(f"   Available tools: {[tool.name for tool in tools]}")

In [ ]:
# Step 3: Get a prompt from LangChain Hub
# This prompt is designed for tool-calling agents

prompt = hub.pull("hwchase17/openai-functions-agent")

print("✅ Prompt loaded from LangChain Hub!")
print(f"   Template variables: {prompt.input_variables}")

In [ ]:
# Step 4: Create the agent

agent = create_tool_calling_agent(llm, tools, prompt)

print("✅ Agent created!")

In [ ]:
# Step 5: Create the Agent Executor
# This orchestrates the ReAct loop

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Show the agent's thought process
    max_iterations=5,  # Prevent infinite loops
)

print("✅ Agent Executor ready!")

In [ ]:
# Step 6: Run the agent!

print("\n" + "="*60)
print("🚀 RUNNING AGENT")
print("="*60 + "\n")

result = agent_executor.invoke({
    "input": "What are the latest developments in AI agents as of 2024?"
})

print("\n" + "="*60)
print("📋 FINAL ANSWER")
print("="*60)
print(result["output"])

### 🎯 Key Takeaways

1. **LLM** is the reasoning engine
2. **Tools** extend what the agent can do
3. **Prompts** guide agent behavior
4. **AgentExecutor** runs the ReAct loop
5. **verbose=True** shows the thought process

---

## 5. Custom Tools with @tool Decorator

The real power of agents comes from custom tools. The `@tool` decorator is the simplest way to create them.

**Critical Insight**: The agent **only sees the function name and docstring**. A clear docstring is the most important part of a tool!

In [ ]:
from langchain.tools import tool

# Tool 1: Simple calculator
@tool
def calculator(expression: str) -> str:
    """Evaluates a mathematical expression and returns the result.
    Use this for any math calculations like addition, subtraction,
    multiplication, division, or complex expressions.
    
    Args:
        expression: A mathematical expression like '2+2' or '10*5+3'
    
    Returns:
        The result of the calculation as a string.
    """
    try:
        # Note: eval is for demo - use a safe parser in production!
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

print("✅ Calculator tool created!")
print(f"   Name: {calculator.name}")
print(f"   Description: {calculator.description[:50]}...")

In [ ]:
# Tool 2: Get current date/time
@tool
def get_current_datetime() -> str:
    """Returns the current date and time. Use this when you need to know
    what day it is, the current time, or any date-related information.
    """
    from datetime import datetime
    now = datetime.now()
    return f"Current date and time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

print("✅ DateTime tool created!")

In [ ]:
# Tool 3: File reader (with safety checks)
@tool
def read_file(filename: str) -> str:
    """Reads and returns the contents of a file.
    Use this to get information from local text files.
    
    Args:
        filename: The name or path of the file to read.
    
    Returns:
        The contents of the file, or an error message if the file doesn't exist.
    """
    try:
        with open(filename, 'r') as f:
            content = f.read()
            # Truncate if too long
            if len(content) > 2000:
                content = content[:2000] + "\n... [truncated]"
            return content
    except FileNotFoundError:
        return f"Error: File '{filename}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"

print("✅ File reader tool created!")

In [ ]:
# Create an agent with custom tools

custom_tools = [calculator, get_current_datetime, read_file]

custom_agent = create_tool_calling_agent(llm, custom_tools, prompt)
custom_executor = AgentExecutor(
    agent=custom_agent,
    tools=custom_tools,
    verbose=True,
)

print("✅ Custom tool agent ready!")
print(f"   Tools: {[t.name for t in custom_tools]}")

In [ ]:
# Test the custom tools

print("\n" + "="*60)
print("🧮 TESTING CALCULATOR TOOL")
print("="*60 + "\n")

result = custom_executor.invoke({
    "input": "What is 125 multiplied by 8, then add 42?"
})
print(f"\n📋 Answer: {result['output']}")

In [ ]:
# Test datetime tool

print("\n" + "="*60)
print("📅 TESTING DATETIME TOOL")
print("="*60 + "\n")

result = custom_executor.invoke({
    "input": "What day of the week is it today?"
})
print(f"\n📋 Answer: {result['output']}")

---

## 6. Structured Tools with Pydantic

For more complex tools with multiple parameters and validation, use Pydantic models.

In [ ]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import Optional

# Define input schema with Pydantic
class WeatherInput(BaseModel):
    """Input for weather lookup."""
    city: str = Field(description="The city to get weather for")
    units: Optional[str] = Field(
        default="fahrenheit",
        description="Temperature units: 'celsius' or 'fahrenheit'"
    )

def get_weather(city: str, units: str = "fahrenheit") -> str:
    """Get weather for a city (simulated)."""
    # Simulated weather data
    weather_data = {
        "new york": {"temp_f": 72, "temp_c": 22, "condition": "Partly Cloudy"},
        "london": {"temp_f": 59, "temp_c": 15, "condition": "Rainy"},
        "tokyo": {"temp_f": 68, "temp_c": 20, "condition": "Sunny"},
        "paris": {"temp_f": 64, "temp_c": 18, "condition": "Cloudy"},
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        temp = data["temp_c"] if units == "celsius" else data["temp_f"]
        unit_symbol = "°C" if units == "celsius" else "°F"
        return f"Weather in {city}: {temp}{unit_symbol}, {data['condition']}"
    else:
        return f"Weather data not available for {city}"

# Create the structured tool
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Get current weather for a city. Can return temperature in celsius or fahrenheit.",
    args_schema=WeatherInput,
)

print("✅ Structured weather tool created!")
print(f"   Schema: city (required), units (optional)")

In [ ]:
# Another structured tool example: Task Manager

class TaskInput(BaseModel):
    """Input for task operations."""
    action: str = Field(description="Action: 'add', 'list', or 'complete'")
    task: Optional[str] = Field(default=None, description="Task description (for add action)")
    task_id: Optional[int] = Field(default=None, description="Task ID (for complete action)")

# Simple in-memory task storage
tasks = []

def manage_tasks(action: str, task: str = None, task_id: int = None) -> str:
    """Manage a simple task list."""
    global tasks
    
    if action == "add" and task:
        tasks.append({"id": len(tasks) + 1, "task": task, "done": False})
        return f"Added task: '{task}' (ID: {len(tasks)})"
    
    elif action == "list":
        if not tasks:
            return "No tasks in the list."
        result = "Tasks:\n"
        for t in tasks:
            status = "✅" if t["done"] else "⬜"
            result += f"  {status} [{t['id']}] {t['task']}\n"
        return result
    
    elif action == "complete" and task_id:
        for t in tasks:
            if t["id"] == task_id:
                t["done"] = True
                return f"Marked task {task_id} as complete."
        return f"Task {task_id} not found."
    
    else:
        return "Invalid action. Use 'add', 'list', or 'complete'."

task_tool = StructuredTool.from_function(
    func=manage_tasks,
    name="task_manager",
    description="Manage tasks: add new tasks, list all tasks, or mark tasks complete.",
    args_schema=TaskInput,
)

print("✅ Task manager tool created!")

In [ ]:
# Test structured tools

structured_tools = [weather_tool, task_tool, calculator]
structured_agent = create_tool_calling_agent(llm, structured_tools, prompt)
structured_executor = AgentExecutor(
    agent=structured_agent,
    tools=structured_tools,
    verbose=True,
)

print("\n" + "="*60)
print("🌤️ TESTING WEATHER TOOL")
print("="*60 + "\n")

result = structured_executor.invoke({
    "input": "What's the weather in Tokyo in celsius?"
})
print(f"\n📋 Answer: {result['output']}")

---

## 7. Agent Memory & Chat History

By default, agents are stateless. To have conversations, we need to manage chat history.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# Create a prompt with memory placeholder
memory_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to tools.
    Use the tools when needed to help answer questions.
    Be conversational and remember the context of our discussion."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

print("✅ Memory-enabled prompt created!")

In [ ]:
# Create memory-enabled agent
memory_tools = [calculator, get_current_datetime]
memory_agent = create_tool_calling_agent(llm, memory_tools, memory_prompt)
memory_executor = AgentExecutor(
    agent=memory_agent,
    tools=memory_tools,
    verbose=True,
)

# Initialize chat history
chat_history = []

print("✅ Memory-enabled agent ready!")

In [ ]:
# Conversation Turn 1
print("\n" + "="*60)
print("💬 CONVERSATION TURN 1")
print("="*60 + "\n")

input1 = "My budget for this month is $5000. I've spent $1200 on rent. How much do I have left?"

result1 = memory_executor.invoke({
    "input": input1,
    "chat_history": chat_history,
})

# Update chat history
chat_history.extend([
    HumanMessage(content=input1),
    AIMessage(content=result1["output"]),
])

print(f"\n📋 Answer: {result1['output']}")

In [ ]:
# Conversation Turn 2 (references previous context)
print("\n" + "="*60)
print("💬 CONVERSATION TURN 2 (with memory)")
print("="*60 + "\n")

input2 = "If I spend another $800 on groceries, what's my remaining budget?"

result2 = memory_executor.invoke({
    "input": input2,
    "chat_history": chat_history,
})

# Update chat history
chat_history.extend([
    HumanMessage(content=input2),
    AIMessage(content=result2["output"]),
])

print(f"\n📋 Answer: {result2['output']}")

In [ ]:
# Conversation Turn 3 (more context reference)
print("\n" + "="*60)
print("💬 CONVERSATION TURN 3")
print("="*60 + "\n")

input3 = "What percentage of my original budget have I spent so far?"

result3 = memory_executor.invoke({
    "input": input3,
    "chat_history": chat_history,
})

print(f"\n📋 Answer: {result3['output']}")

---

## 8. Different Agent Types

LangChain supports multiple agent types for different use cases.

In [ ]:
print("""
┌─────────────────────────────────────────────────────────────────┐
│                    LangChain Agent Types                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   TOOL CALLING AGENT (Recommended)                             │
│   └── Uses native function calling                             │
│   └── Works with: GPT-4, Claude, Gemini                       │
│   └── Most reliable tool selection                            │
│                                                                 │
│   REACT AGENT                                                  │
│   └── Classic Reason + Act pattern                            │
│   └── Works with any LLM                                      │
│   └── More verbose output                                     │
│                                                                 │
│   OPENAI FUNCTIONS AGENT                                       │
│   └── OpenAI-specific function calling                        │
│   └── Legacy, prefer tool_calling_agent                       │
│                                                                 │
│   STRUCTURED CHAT AGENT                                        │
│   └── For complex multi-input tools                           │
│   └── JSON-based tool calls                                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
""")

In [ ]:
# Example: ReAct Agent (works with any LLM)
from langchain.agents import create_react_agent
from langchain import hub

# Get ReAct prompt
react_prompt = hub.pull("hwchase17/react")

# Create ReAct agent
react_agent = create_react_agent(llm, custom_tools, react_prompt)
react_executor = AgentExecutor(
    agent=react_agent,
    tools=custom_tools,
    verbose=True,
    handle_parsing_errors=True,  # Gracefully handle errors
)

print("✅ ReAct agent created!")

---

## 9. Streaming & Real-time Output

For better UX, you can stream agent output in real-time.

In [ ]:
# Streaming agent output

print("\n" + "="*60)
print("📡 STREAMING AGENT OUTPUT")
print("="*60 + "\n")

# Stream events from the agent
for event in custom_executor.stream({"input": "What is 15 factorial?"}):
    # Print each event type
    if "output" in event:
        print(f"\n🎯 Final Output: {event['output']}")
    elif "intermediate_step" in event:
        action = event["intermediate_step"][0]
        print(f"\n🔧 Tool: {action.tool}")
        print(f"   Input: {action.tool_input}")

---

## 10. Error Handling & Debugging

Robust agents need proper error handling.

In [ ]:
# Agent with comprehensive error handling

robust_executor = AgentExecutor(
    agent=custom_agent,
    tools=custom_tools,
    verbose=True,
    max_iterations=5,           # Prevent infinite loops
    max_execution_time=30,      # Timeout in seconds
    handle_parsing_errors=True, # Gracefully handle parse errors
    return_intermediate_steps=True,  # Return all steps for debugging
)

print("✅ Robust agent executor configured!")
print("   - Max iterations: 5")
print("   - Timeout: 30 seconds")
print("   - Parse error handling: Enabled")

In [ ]:
# Example with intermediate steps for debugging

print("\n" + "="*60)
print("🔍 DEBUGGING: VIEWING INTERMEDIATE STEPS")
print("="*60 + "\n")

result = robust_executor.invoke({
    "input": "What is 100 divided by 4, and what time is it?"
})

print("\n📊 INTERMEDIATE STEPS:")
for i, step in enumerate(result.get("intermediate_steps", [])):
    action, observation = step
    print(f"\n  Step {i+1}:")
    print(f"    Tool: {action.tool}")
    print(f"    Input: {action.tool_input}")
    print(f"    Result: {observation}")

---

## 11. Real-World Patterns

Common patterns for building production agents.

In [ ]:
# Pattern 1: Research Agent with multiple sources

@tool
def search_documentation(query: str) -> str:
    """Search internal documentation for information.
    Use this for questions about company policies, procedures, or technical docs.
    """
    # Simulated documentation search
    docs = {
        "vacation": "Employees get 20 days of paid vacation per year.",
        "remote": "Remote work is allowed 3 days per week.",
        "expense": "Expense reports must be submitted within 30 days.",
    }
    for key, value in docs.items():
        if key in query.lower():
            return f"Documentation: {value}"
    return "No relevant documentation found."

@tool
def search_knowledge_base(query: str) -> str:
    """Search the knowledge base for FAQs and common solutions.
    Use this for general questions and troubleshooting.
    """
    # Simulated KB search
    kb = {
        "password": "To reset your password, visit the IT portal.",
        "vpn": "VPN issues? Try disconnecting and reconnecting.",
        "printer": "For printer issues, submit a ticket to facilities.",
    }
    for key, value in kb.items():
        if key in query.lower():
            return f"Knowledge Base: {value}"
    return "No FAQ found for this topic."

# Combine tools for a research agent
research_tools = [search_documentation, search_knowledge_base]

print("✅ Research agent tools created!")

In [ ]:
# Pattern 2: Data Analysis Agent

import json

@tool
def query_database(sql_query: str) -> str:
    """Execute a SQL-like query on the sales database.
    Supports: SELECT, COUNT, SUM, AVG on sales data.
    Available columns: date, product, quantity, price, region
    """
    # Simulated database
    if "total" in sql_query.lower() or "sum" in sql_query.lower():
        return "Query result: Total sales = $1,234,567"
    elif "count" in sql_query.lower():
        return "Query result: 5,432 transactions"
    elif "avg" in sql_query.lower():
        return "Query result: Average order value = $227.45"
    else:
        return "Query executed. 100 rows returned."

@tool
def generate_chart(chart_type: str, data_description: str) -> str:
    """Generate a chart visualization.
    Types: 'bar', 'line', 'pie'
    Returns a description of the generated chart.
    """
    return f"Generated {chart_type} chart showing: {data_description}. Chart saved to output/chart.png"

print("✅ Data analysis tools created!")

In [ ]:
# Pattern 3: Customer Service Agent

@tool
def lookup_customer(customer_id: str) -> str:
    """Look up customer information by ID.
    Returns: name, email, membership status, account balance.
    """
    # Simulated customer lookup
    customers = {
        "C001": {"name": "John Doe", "email": "john@example.com", "status": "Gold", "balance": 150.00},
        "C002": {"name": "Jane Smith", "email": "jane@example.com", "status": "Silver", "balance": 75.50},
    }
    if customer_id in customers:
        return json.dumps(customers[customer_id], indent=2)
    return f"Customer {customer_id} not found."

@tool
def create_support_ticket(customer_id: str, issue: str, priority: str) -> str:
    """Create a support ticket for a customer.
    Priority: 'low', 'medium', 'high'
    Returns the ticket ID.
    """
    import random
    ticket_id = f"TKT-{random.randint(10000, 99999)}"
    return f"Ticket {ticket_id} created for customer {customer_id}. Priority: {priority}. Issue: {issue}"

print("✅ Customer service tools created!")

---

## 12. Best Practices & Common Pitfalls

### ✅ Best Practices

1. **Write clear tool docstrings** - The agent only sees the docstring!
2. **Use type hints** - Help the agent understand parameter types
3. **Return meaningful errors** - Don't just fail silently
4. **Set reasonable limits** - max_iterations, max_execution_time
5. **Test tools in isolation** - Before adding to an agent

### ❌ Common Pitfalls

1. **Vague docstrings** - "Does stuff" vs "Calculates the sum of two numbers"
2. **Too many tools** - Agent gets confused with 10+ tools
3. **No error handling** - Crashes break the whole agent
4. **Missing context** - Agent needs to know WHEN to use tools
5. **Forgetting verbose=True** - Makes debugging impossible

In [ ]:
# Production-ready agent configuration template

def create_production_agent(tools: list, system_message: str = None):
    """Create a production-ready agent with best practices."""
    
    # Default system message
    if system_message is None:
        system_message = """You are a helpful assistant with access to tools.
        Use tools when needed to provide accurate information.
        If you're unsure, say so rather than guessing."""
    
    # Create prompt with memory support
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create agent
    agent = create_tool_calling_agent(llm, tools, prompt)
    
    # Create executor with safeguards
    executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=False,  # Disable in production, log instead
        max_iterations=10,
        max_execution_time=60,
        handle_parsing_errors=True,
        return_intermediate_steps=False,
    )
    
    return executor

print("✅ Production agent template created!")

---

## 13. Conclusion & Next Steps

### What You've Learned

| Topic | Key Takeaway |
|-------|-------------|
| ReAct Loop | Agents reason, act, and observe |
| Tools | Extend agent capabilities with functions |
| @tool Decorator | Simplest way to create tools |
| Structured Tools | For complex, multi-parameter tools |
| Memory | Chat history enables conversations |
| Debugging | verbose=True is your best friend |

### Next Steps

1. **Practice**: Build an agent for your specific use case
2. **Explore LangGraph**: For complex stateful workflows
3. **Compare frameworks**: See how CrewAI/AutoGen differ
4. **Production**: Add logging, monitoring, and error handling

### Resources

- [LangChain Documentation](https://python.langchain.com/docs/)
- [LangChain Agents Guide](https://python.langchain.com/docs/modules/agents/)
- [LangSmith](https://smith.langchain.com/) - For debugging and monitoring

---

**Congratulations!** You've completed the LangChain Agents Zero to Hero guide! 🎉